In [ ]:
import os

os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

from mirrorverse.utils import read_data_w_cache

from haven.db import write_data

import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import h3

In [ ]:
sql = '''
select 
    f.*,
    o.parallel_to_shelf,
    o.toward_coast,
    o.toward_shelf,
    cos(o.angle) as cos_angle,
    sin(o.angle) as sin_angle,
    o.angle,
    c.coast_distance,
    c.shelf_distance,
    cos(s.season_radians) as cos_season_radians,
    sin(s.season_radians) as sin_season_radians,
    s.season_radians
from 
    movement_model_features_m9_a2 f
    inner join options_w_angles o 
        on f.origin_h3_index = o.h3_index
        and f.h3_index = o.neighbor 
    inner join coastal_info c
        on f.origin_h3_index = c.h3_index
    inner join season_radians s
        on to_unixtime(f.time) = s.epoch
'''
data = read_data_w_cache(sql)
print(data.shape)
data.head()

In [ ]:
df = data[(data['_individual'] == 10) & (data['_decision'] == 10)]
df['lat'] = df['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
df['lon'] = df['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])

px.scatter_geo(
    df,
    lat='lat',
    lon='lon',
    color='toward_shelf',
).show()

In [ ]:
write_data(
    data, 'movement_model_features_10', ['_train']
)

In [ ]:
data.columns